# Raspagem de dados web com Python

Este notebook expõe os conceitos básicos de raspagem de dados web (web scraping) e propõe alguns exercícios.
Utilizaremos conceitos pythônicos como funções e controle de fluxo, além de conceitos da Internet como o protocolo HTTP, URLs e de componentes fundamentias da web, como HTML, CSS, JavaScript etc. 

## Como funciona a web?

### Internet enquanto sistema global de redes de computadores interconectadas
#### Redes de computadores
![A rede mundial de computadores](internet.png "Internet")

#### Infraestrutura
![Cabos submarinos que conectam os computadores](internet2.png "Infraestrutura da Internet")

#### Os URLs e o Protocolo HTTP
* URL: Uniform Resource Layer -> endereço web
* HTTP: Hypertext Transfer Protocol -> fundação da comunicação de dados na web

![O protocolo HTTP e o URL sendo usado no Browser](http.png "HTTP/URL no Browser")





### Como o navegador transforma os dados recebidos via HTTP em elementos visuais?

#### O código-fonte dos websites: HTML, CSS e JavaScript

Exemplo da página [http://pythonscraping.com/pages/page1.html](http://pythonscraping.com/pages/page1.html)

```html
<html>
<head>
<title>A Useful Page</title>
</head>
<body>
<h1>An Interesting Title</h1>
<div>
Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>
</body>
</html>
```

#### Um exemplo mais complexo, com CSS: [https://quotes.toscrape.com](https://quotes.toscrape.com); e outro, com JavaScript: [https://www.globo.com/](https://www.globo.com/)
É preciso clicar com o botão direito na página e clicar em `Exibir código-fonte`.

## Do navegador ao código: como ler a web com Python?

In [ ]:
!pip install requests
!pip install bs4

### `requests`: fazendo pedidos HTTP com Python

In [ ]:
import requests

r = requests.get('http://pythonscraping.com/pages/page1.html')

In [ ]:
# biblioteca interna ao Python para fazer prints mais bonitos
from pprint import pprint

pprint(r.content)

### `BeautifulSoup`: transformando HTML em dados estruturados

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
soup.prettify()

#### Navegando pela árvore do HTML

In [ ]:
soup.head

In [ ]:
soup.title

In [ ]:
soup.h1

In [ ]:
soup.div

## Raspando a web: um exemplo introdutório

Vamos unir as explicações desenvolvidas acima aos nossos conhecimentos de Python para raspar a seguinte página: [https://quotes.toscrape.com/](https://quotes.toscrape.com/). Essa página foi criada pela empresa ScrapingHub, desenvolvedora da biblioteca de web scraping avançado `Scrapy`, com o objetivo de introduzir iniciantes à raspagem de dados.

In [ ]:
r = requests.get("https://quotes.toscrape.com/")
soup = BeautifulSoup(r.content, 'html.parser')

#### Como obter a citação do Einstein?

In [ ]:
soup.span.text

In [ ]:
soup.small.text

#### Os métodos `find` e `find_all` do `BeautifulSoup`

In [ ]:
soup.find('span', class_='text').text

In [ ]:
soup.find('small', class_="author").text

#### Obtendo uma lista de elementos com o find_all

In [ ]:
soup.find_all('span')

In [ ]:
elements = soup.find_all('span')
elements

In [ ]:
elements = [elements.text for elements in soup.find_all('span', class_='text')]
elements

#### Construindo uma `list` de citações e autores

In [ ]:
authors = [author.text for author in soup.find_all('small', class_='author')]
quotes = [quote.text for quote in soup.find_all('span', class_='text')]

data = list(zip(authors, quotes))
data

#### Construindo uma função que retorna os registros de citações de uma página (autores, citação etc.)

In [ ]:
def get_quote_records(soup):
    quotes = [quote.text for quote in soup.find_all("span", class_="text")]
    authors = [author.text for author in soup.find_all("small", class_="author")]

    data = list(zip(authors, quotes))

    return data


get_quote_records(soup)


In [ ]:
count = 1
all_data = []

while True:
    # constrói o objeto `soup`
    r = requests.get(f"https://quotes.toscrape.com/page/{count}")
    soup = BeautifulSoup(r.content, "html.parser")

    # anexa os dados à list `data`
    page_data = get_quote_records(soup)
    all_data += page_data

    # incrementa o contador
    count += 1

    # condição de parada: quando não há mais dados
    if page_data == []:
        break

# mostra os dados
print(all_data)


#### Adicionando os dados de `tags`

In [ ]:
tag_divs = soup.find_all("div", class_="tags")

all_tags = []
for tag_div in tag_divs:
    tags = tag_div.find_all("a")
    tags = [tag.text for tag in tags]
    all_tags.append(tags)

all_tags

#### Encapsulando tudo em fuções

In [ ]:
def get_tags(soup):
    tag_divs = soup.find_all("div", class_="tags")

    all_tags = []
    for tag_div in tag_divs:
        tags = tag_div.find_all("a")
        tags = [tag.text for tag in tags]
        all_tags.append(tags)

    return all_tags


def get_quote_records(soup):
    quotes = [quote.text for quote in soup.find_all("span", class_="text")]
    authors = [author.text for author in soup.find_all("small", class_="author")]
    tags = get_tags(soup)

    data = list(zip(authors, quotes, tags))

    return data


get_quote_records(soup)


In [ ]:
def get_all_quotes(soup):
    count = 1
    all_data = []

    while True:
        # constrói o objeto `soup`
        r = requests.get(f"https://quotes.toscrape.com/page/{count}")
        soup = BeautifulSoup(r.content, "html.parser")

        # anexa os dados à list `data`
        page_data = get_quote_records(soup)
        all_data += page_data

        # incrementa o contador
        count += 1

        # condição de parada do loop: quando não há mais dados
        if page_data == []:
            break

    return all_data


get_all_quotes(soup)


## Raspando páginas que demandam interação com o browser

Nessa seção utilizaremos a biblioteca `helium`, que nos permite interagir com as páginas da web diretamente do Python. Com ele é possível clicar em botões, escrever dados em formulários e muito mais de uma forma muito mais simples que o mais conhecido `selenium`.

#### Fazendo login na página com `helium`

In [ ]:
!pip install helium

In [ ]:
from helium import (
    start_chrome,
    write,
    click,
    press,
    TAB,
    ENTER,
    kill_browser,
)

driver = start_chrome("https://quotes.toscrape.com/")
click("Login")
write("a", into="Username")
press(TAB)
write("b", into="Password")
press(ENTER)


In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

soup.title

In [ ]:
quotes_data = get_all_quotes(soup)
quotes_data


## Exportando tudo para uma tabela em CSV com `pandas`

In [ ]:

data = dict(zip(['authors', 'quotes', 'tags'], zip(*quotes_data)))
data

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df

In [ ]:
df.to_csv('scraped_data.csv', index=False)

## Usando o `pandas` para raspar tabelas de websites

In [ ]:
!pip install html5lib

In [ ]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_Copa_Libertadores_finals', flavor='html5lib')

In [ ]:
data[2].head(15)

In [ ]:
data[3].head(15)

In [ ]:
data[4]

# Referências

## Tecnologias da Web
* [https://en.wikipedia.org/wiki/Internet]()
* [https://en.wikipedia.org/wiki/URL]()
* [https://en.wikipedia.org/wiki/World_Wide_Web]()
* [https://en.wikipedia.org/wiki/HTML]()
* [https://en.wikipedia.org/wiki/CSS]()
* [https://en.wikipedia.org/wiki/JavaScript]()

## Raspagem de dados com Python
* MITCHELL, R. [Web Scraping with Python](https://www.oreilly.com/library/view/web-scraping-with/9781491985564/). 2. ed. Sebastopol, CA, O’Reilly Media, Inc., 2018. 
* 